Yolov5の公式Tutorialはこちらを参照してください．
https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb

ただ，以下のコードをそのまま実行していっていただければ基本的には大丈夫だと思います．

In [ ]:
# Yolov5をインストールする

!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# 自分のGoogleドライブと接続します．リンクから飛んでパスワードをコピーしてこのセルの下に出てくる入力欄の中にペーストしてEnterを押したらマウントしてくれます．
# 大量の画像データは時間はかかりますが，一旦自分のパソコンから自分のGoogleドライブにアップロードして，Colabで使う方がColabの計算量も食わないし，色々と効率的です．

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 学習画像を学習する
# --imgは画像の幅・高さのうち大きい方のサイズです．大きさを間違えて入力しても特に変化がなかったのであまり気にする必要はないと思います．
# --batchは一度に何枚の画像を学習するかです．--epochsは--batchで指定した枚数を学習することを一回とした時に何回その学習を行うかです．以下の--batch 10 --epochs 100という場合だと一度に10枚学習してそれを100回繰り返すという意味なので，学習画像が全部で100枚しかないとすると1枚の画像につき10回は繰り返し学習をすることになります．
# --batchを大きくしすぎると一度の計算量が大きくなってColabがリミットを計算量のリミットを超過してしばらく使えなくなってしまいます．小さくしすぎると計算時間がかかるし，学習が非効率的になってしまいます．
# --epochを小さくしすぎると最適値に到達するまでに必要な学習の回数に足りなくなってしまいます．大きくしすぎると最適値に到達した後にも永遠と学習し続けてしまうので余計な時間がかかってしまいます．
# 私がやったときは学習画像が100枚ぐらいだったので--batch 10 --epochs 100ぐらいがちょうど良かったです．
# --dataは学習の際に必要な情報（学習画像があるディレクトリのパスだったりとかの情報）が書かれているYamlファイルのパスを指定します．train.yamlのパスを書いてください．
# --weightsはyolov5s.pt等を指定しておけばOKだと思います．
# --cacheは実行の際の読み取ったものをキャッシュしてくれるとかなんとからしいですが，特に書いても書かなくても問題ないと思います．書いておいたら再度学習するときにより早く学習できる感じです．

# うまく実行できた場合は学習の最後に以下の文と同じようなものが出力されると思います．
##################################################################################################################################
# 100 epochs completed in 0.273 hours.
# Optimizer stripped from runs/train/exp/weights/last.pt, 14.4MB  ←last.ptとは，一番最後に学習をしたときに得られる重みが書かれているファイルです．
# Optimizer stripped from runs/train/exp/weights/best.pt, 14.4MB  ←best.ptとは，学習をして得られた重みの中で一番最適な重みが書かれているファイルです．最適な重みが書かれたものなのでlast.ptではなく，当然こちらの重みファイルの方を以下では使っていくことになります．ここでは「runs/train/exp/weights/best.pt」がbest.ptのパスです．

# Validating runs/train/exp/weights/best.pt...
# Fusing layers... 
# Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 5/5 [00:04<00:00,  1.09it/s]
#                  all        100        143      0.812      0.818      0.891      0.485
# Results saved to runs/train/exp
##################################################################################################################################

!python train.py --img 600 --batch 10 --epochs 100 --data /content/drive/MyDrive/train.yaml --weights yolov5s.pt --cache

In [ ]:
# 重み（best.pt）を自分のGoogleドライブに保存
# 上のセルで得られたbest.ptは念の為自分のGoogleドライブに保存した方がいいです．そうすればまた推論するときに再度学習をしてbest.ptを生成し直す必要がなくなります．
# 「%cp best.ptのパス 移動先のパス」で，best.ptが指定した移動先にコピーされます．best.ptのパスは上のセルで出力されたbest.ptのパスをコピペしてください．移動先のパスは自分が移動させたい場所のパスに合わせて変えてください．

%cp runs/train/exp/weights/best.pt /content/drive/MyDrive/

In [ ]:
# 推論（検出）する
# 上記の学習で得た最適値の重み（best.pt）を使って，テスト画像を検出します
# このセルでは検出画像を吐き出すだけなので，精度を知りたいときはこのセルの一個下のセルを使ってください．
# --weightsにはbest.ptのパスを貼ってください．
# --imgは学習のときと同じく画像のサイズです．
# --confは閾値です．つまりどれぐらいの確度のものを検出するかということです．やってみるとわかりますが閾値を下げると大量に物体を検出してしまいます．逆にあげすぎると何も検出しなくなります．この数値はいじって適度に検出してくれるぐらいの数値に調整してください．僕の場合は0.1ぐらいでちょうど良かったです．
# --sourceにはテスト画像が入っているGoogleドライブのディレクトリのパスを入れてください．

# 最後に以下と同じような結果が出力されると思います．
##################################################################################################################################
# Speed: 0.5ms pre-process, 23.4ms inference, 0.9ms NMS per image at shape (1, 3, 608, 608)
# Results saved to runs/detect/exp  ←検出画像がこのパスに保存されたと言っています．ここでは検出画像が入っているディレクトリのパスは「runs/detect/exp」です．
##################################################################################################################################

!python val.py --weights runs/train/exp/weights/best.pt --img 600 --conf 0.1 --source /content/drive/MyDrive/dataset/test/images/

In [ ]:
# 検出画像を自分のGoogleドライブに保存
# 「%cp 検出画像が保存されているディレクトリのパス 移動先のパス」で，検出画像が保存されているディレクトリが指定した移動先にコピーされます．検出画像が保存されているディレクトリのパスは上のセルで出力された検出画像が保存されているディレクトリのパスをコピペしてください．移動先のパスは自分が移動させたい場所のパスに合わせて変えてください．

%cp runs/detect/exp /content/drive/MyDrive/

In [ ]:
# 精度評価
# このセルでは最適値の重み（best.pt）を使って，テスト画像を検出したときの精度（mAP）を出します．客観的に精度を数値化できるので精度比較の際の指標として使います．
# mAPについては　https://pystyle.info/how-to-calculate-object-detection-metrics-map/　または　https://qiita.com/cv_carnavi/items/08e11426e2fac8433fed　の説明がわかりやすいです．ただ，Yolov5が自動でmAPの数値を出してくれるので左記の記事にならって自分で計算する必要はありません．
# --weightsにはbest.ptのパスを入れてください．
# --imgは学習のときと同じく画像のサイズです．
# # --dataは精度評価の際に必要な情報（テスト画像があるディレクトリのパスだったりとかの情報）が書かれているYamlファイルのパスを指定します．test.yamlのパスを書いてください．

# 最後に以下と同じような結果が出力されると思います．この出力部分はスクショしておいてください．なぜかこの時にしか見れないので出力結果全体をスクショしておいてGoogleドライブに保存しておいた方がいいです．
# mAP@.5 は閾値が0.5の時のmAPの数値．
# mAP@.5:.95は閾値が0.5, 0.55, 0.6, 0.65 ... 0.85, 0.9, 0.95のそれぞれの時のmAPの数値の平均値．
##################################################################################################################################
# val: data=/content/drive/MyDrive/test.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=600, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False
# YOLOv5 🚀 v6.0-28-g79d8f1f torch 1.9.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)

# Fusing layers... 
# Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
# WARNING: --img-size 600 must be multiple of max stride 32, updating to 608
# val: Scanning '/content/drive/MyDrive/dataset/test/labels.cache' images and labels... 1055 found, 0 missing, 0 empty, 0 corrupted: 100% 1055/1055 [00:00<?, ?it/s]
#                Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 33/33 [02:22<00:00,  4.31s/it]
#                  all       1055       1652      0.427      0.326      0.287     0.0991  ←このmAP@.5とmAP@.5:.95の数値がmAPの数値です．
# Speed: 0.4ms pre-process, 15.6ms inference, 2.0ms NMS per image at shape (32, 3, 608, 608)
# Results saved to runs/val/exp3  ←結果がこのディレクトリに保存されたと言っています．ここでは結果が保存されているディレクトリのパスは「runs/val/exp3」です．
##################################################################################################################################

!python val.py --weights runs/train/exp/weights/best.pt --img 600 --data /content/drive/MyDrive/test.yaml

In [ ]:
# 精度評価の際の結果を自分のGoogleドライブに保存
# 「%cp 結果が保存されているディレクトリのパス 移動先のパス」で，結果が保存されているディレクトリが指定した移動先にコピーされます．結果が保存されているディレクトリのパスは上のセルで出力された結果が保存されているディレクトリのパスをコピペしてください．移動先のパスは自分が移動させたい場所のパスに合わせて変えてください．

%cp runs/val/exp3 /content/drive/MyDrive/